## Factuality Analysis of CommunityLM models

### Environment Setup

In [ ]:
import pandas as pd
import numpy as np
import csv
import json
import nltk
nltk.download('punkt_tab')

from llments.eval.factscore.factscorer import FactScorer

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install rank_bm25

In [ ]:
!pip install torch -U

In [ ]:
!python -m llments.eval.factscore.download_data --data_dir /factscore_data

In [ ]:
fs10 = FactScorer(model_name="retrieval+ChatGPT",
                 data_dir="/factscore_data",
                 model_dir="/factscore_data",
                 cache_dir="/factscore_data",
                 openai_key="key.txt",
                 cost_estimate="consider_cache",
                 abstain_detection_type=None,
                 batch_size=256)

### Registering Knowledge Source

To do the factuality analysis on the respones of the CommunityLM responses, we first need to register them as a knowledge source. We register four knowledge sources based on:
1. Responses for Democratic politicians using the Democratic model
2. Responses for Republican politicians using the Democratic model
3. Responses for Democratic politicians using the Republican model
4. Responses for Republican politicians using the Republican model

The following code splits the responses of the Democratic model into responses for Democratic politicians and Republican politicians. Then it registers two knowledge sources, for Democrats using the Democratic model and Republicans using the Democratic model. The same process can be applied for the other knowledge sources.



In [ ]:
dem_data = []
rep_data = []

with open('/factscore_data/democratic-final.csv', 'r') as file:     # Responses of the Democratic model for politicians of both parties
   reader = csv.reader(file)
   next(reader)  # Skip header
   for row in reader:
       party, politician, response = row
       entry = {
           "title": politician.strip('*'),
           "text": response
       }
       if 'Democrats' in party:
           dem_data.append(entry)
       elif 'Republicans' in party:
           rep_data.append(entry)

# Write Democrats JSONL
with open('/factscore_data/democrats-democratic-final-model.jsonl', 'w') as f:    #jsonl for knowledge source of democratic politicians
   for entry in dem_data:
       f.write(json.dumps(entry) + '\n')

# Write Republicans JSONL
with open('/factscore_data/republicans-democratic-final-model.jsonl', 'w') as f:  #jsonl for knowledge source of republican politicians
   for entry in rep_data:
       f.write(json.dumps(entry) + '\n')

In [ ]:
# Knowledge source using responses of Democratic model for Democratic politicians

fs10.register_knowledge_source("democrats-democratic-final-model",
                             data_path="/factscore_data/democrats-democratic-final-model.jsonl",
                             db_path="/factscore_data/democrats-democratic-final-model.db")

In [ ]:
# Knowledge source using responses of Democratic model for Republican politicians

fs10.register_knowledge_source("republicans-democratic-final-model",
                             data_path="/factscore_data/republicans-democratic-final-model.jsonl",
                             db_path="/factscore_data/republicans-democratic-final-model.db")

### FactScore Calculation for CommunityLM Models

To perform the factuality analysis, we calculate the FactScore of both models with respect to politicians of both parties. The code below is an example of how to calulate the FactScore of the Democratic model for Democratic and Republican politicians. The process is used to calculate for the Republican model.

In [ ]:
def get_model_data(model):
  if model == "democratic":
    df = pd.read_csv("/factscore_data/democratic-ks.csv")  # Responses of the Democratic model for politicians of both parties
  else:
    df = pd.read_csv("/factscore_data/republican-ks.csv")  # Responses of the Republican model for politicians of both parties
  return df

In [ ]:
# choose the CommunityLM model for doing the analysis

model="democratic"  # or republican
df = get_model_data(model)
print(df.head())

In [ ]:
democratic_topics = ["Joe Biden", "Bernie Sanders", "Elizabeth Warren", "Chuck Schumer", "Kamala Harris", "Barack Obama", "Pete Buttigieg", "Amy Klobuchar", "Andrew Yang", "Nancy Pelosi"]

republican_topics = ["Donald Trump", "Ron DeSantis", "John McCain", "Ted Cruz", "Nikki Haley", "Lindsey Graham", "Tim Scott", "Mike Pence", "Marco Rubio", "Clarence Thomas"]

In [ ]:
democratic_responses = []

for topic in democratic_topics:
  responses = df.loc[df['Politician'].str.strip().str.lower() == topic.lower(), 'Response']
  combined = ' '.join(responses.head(20).dropna())
  democratic_responses.append(combined)

In [ ]:
republican_responses = []

for topic in republican_topics:
  responses = df.loc[df['Politician'].str.strip().str.lower() == topic.lower(), 'Response']
  combined = ' '.join(responses.head(20).dropna())
  republican_responses.append(combined)

In [ ]:
democratic_outputs = fs10.get_score(democratic_topics, democratic_responses, knowledge_source="democrats-democratic-final-model")
print (democratic_outputs["score"]) # FActScore
print (democratic_outputs["num_facts_per_response"]) # average number of atomic facts per response

In [ ]:
republican_outputs = fs10.get_score(republican_topics, republican_responses, knowledge_source="republicans-democratic-final-model")
print (republican_outputs["score"]) # FActScore
print (republican_outputs["num_facts_per_response"]) # average number of atomic facts per response